In [1]:
import redis
import threading

In [6]:
import redis
# 连接到 Redis
r = redis.Redis(host='localhost', port=6379)

# 订阅频道并处理接收到的消息
def handle_message(message):
    print("Received message:", message)

# 创建订阅对象
p = r.pubsub()

# 订阅频道 "my-channel"
p.subscribe(**{"my-channel": handle_message})

# 启动订阅线程
thread = p.run_in_thread(sleep_time=0.01)

Received message: {'type': 'message', 'pattern': None, 'channel': b'my-channel', 'data': b'Hello, subscribers!'}


In [7]:
# 在适当的时机取消订阅所有频道
thread.stop()

p.unsubscribe()

# 关闭订阅对象
p.close()

In [7]:
# 定义消息处理函数
def message_handler(terminate_event):
    while not terminate_event.is_set():
        message = p.get_message()
        if message and message["type"] == "message":
            handle_message(message["data"])
    print("Thread ended.")

In [8]:
# 创建 Event 对象
terminate_event = threading.Event()
# 创建并启动消息处理线程
message_thread = threading.Thread(target=message_handler, args=(terminate_event,))
message_thread.start()

Received message: b'Hello, subscribers!'
Received message: b'Hello, subscribers!'
Thread ended.


In [9]:
# 模拟一段时间后终止订阅
import time
time.sleep(5)

# 设置终止事件
terminate_event.set()

# 发送唤醒消息以解除阻塞
r.lpush("my-channel", "Wake up!")

# 等待线程结束
message_thread.join()